# Lab 8: Bayesian and Frequentist Takes on Multi-Armed Bandits
Welcome to the eighth DS102 lab! 

The goals of this lab is to implement and gain a better understanding of the pros and cons of the Upper Confidence Bounds and Thompson Sampling algorithms

The code you need to write is commented out with a message "TODO: fill in". There is additional documentation for each part as you go along.


## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

**Submission**: to submit this assignment, rerun the notebook from scratch (by selecting Kernel > Restart & Run all), and then print as a pdf (File > download as > pdf) and submit it to Gradescope.


**This assignment should be completed and submitted before Tuesday November 5, 2019 at 11:59 PM.** 

Write collaborator names here.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.widgets import Button, CheckButtons
from matplotlib import gridspec
import functools
from Bandit_env import BanditEnv, Interactive_UCB_Algorithm,Interactive_TS_Algorithm

In [ ]:
means=[10,9.5,8.5,7.5,7.0,6.5]
variance=1.2
standard_deviations=[np.sqrt(variance) for arm in range(len(means))]
bandit_env=BanditEnv(means,standard_deviations)

# Multi-Armed Bandits

In this lab we will be implementing two of the most common approaches to solving stochastic Multi-Armed Bandit (MAB) problems. We first define the problem and then you will have a chance to implement at the Upper Confidence Bounds and Thompson Sampling algorithms from lecture and analyze their performance.

## Setup:
A MAB problem is a simple setting in which it is easy to analyze the __exploration/exploitation__ tradeoff that is extremely common in machine learning. The setup is as follows:

A MAB instance is a set $\mathcal{A}$ of $K$ arms. Each arm $a \in \mathcal{A}$ is associated with a reward distribution $P_a$ which is unique to that arm. The mean of an arm $a \in \mathcal{A}$ is denoted as $\mu_a=\mathbb{E}_{P_a}[X_a]$. 

At each time $t=1,2,...$, an algorithm that is interacting with a MAB must choose an arm, $A_t \in \mathcal{A}$. The algorithm then receives a reward $X_{A_t,t}$ sampled independently from $P_{A_t}$.

The __goal__ of an algorithm interacting with a MAB instance is to find the arm $A^\ast$ with the highest mean reward $\mu^\ast$ as fast as possible while maintaining performance. That is, it would like to find the arm $A^\ast$ such that:

$$ A^\ast=arg\max_{a \in \mathcal{A}} \mu_a$$

where $\mu^*=\mu_{A^\ast}$.

This is often encoded as wanting to find an algorithm that minimizes the __regret__ over a time horizon $n$. Intuitievly, the regret is the best the algorithm could have done in hindsight if it had known which was the optimal arm. The regret of an algorithm is defined as:

$$ Regret(n)= \sum_{t=1}^n X_{A^\ast,t} -X_{A_t,t}$$


Most of the time, it is simpler to analyze the __pseudo-regret__, which is the mean of the regret.

$$ R(n)= n \mu^* - \mathbb{E}\left[\sum_{t=1}^n X_{A_t,t}\right]$$

### Lab setup:
In this lab, the MAB instances will have a set of arms numbered $0,1,...,K-1$. Each arm $a=0,1,...,K-1$ is associated with a Gaussian reward distribution with mean $\mu_a$ and standard deviation of $\sigma_a=1.2$. To be able to analyze the various algorithms, the optimal arm $A^\ast$ will always be arm $0$, and its mean will always be $\mu^\ast=10$.

By running the following cell, you can interact with a MAB instance of the type we will be using in this lab. You can see the reward distributions as well as the expected cumulative regret you incur when pulling each arm. 

Verify for yourself that explore-then-commit strategies can get stuck pulling the wrong arm.


In [ ]:
# Creates an interactive bandit instance.
#     - Pull an arm by clickling on the colored button
#     - The true means of the distributions are shown with the dashed horizontal lines
#     - Large solid circle is the sample mean of the arm
#     - Small empty circle is a sample from the arm
#     - The reward distribution of each arm is shown on the right and can be toggled on/off by checking the box
#     - Running Pseudo-regret is shown on the bottom and can be toggled on/off by checking the box

# You may need to rerun this cell to restart the gui

%matplotlib notebook
plt.rcParams['figure.figsize']=[9,8]
bandit_env.run_Interactive()

# 1.  The Frequentist Approach: Upper Confidence Bounds

The first algorithm we will analyze is the frequentist take on multi-armed bandits, known as the Upper Confidence Bounds algorithm. 

For each arm $a \in \{ 0,1,...,K-1\}$, you keep track of:

1. $T_a(t)$: the number of times arm $a$ has been pulled up to and including iteration $t$.
2. $X_{a,1},...,X_{a,T_a(t)}$: the samples you have received from arm $a$.

Using this information, you compute an upper confidence bound, $C_a(T_a(t),\delta)$ that encompasses the true mean $\mu_a$ with probability at least $1-\delta$. $C_a(T_a(t),\delta)$, must therefore satisfy:

$$ \mathbb{P}\bigg(\mu_a < C_a(T_a(t),\delta)\bigg) > 1 - \delta.$$

Note that since $C_a(n_a(t),\delta)$ is a function of the samples $X_{a,1},...,X_{a,T_a(t)}$, it is a random variable, and we set the confidence bounds after $0$ samples to $\infty$. That is:

$$C_a(0,\delta)=\infty.$$

The algorithm then pulls, at each round $t$, the arm with the highest upper confidence bound:

$$ A_t=\underset{a \in \{ 0,1,...,K-1\}}{\operatorname{argmax}} C_a(T_a(t),\delta),$$

where ties are broken arbitrarily.

## A. Chebyshev-UCB

In this first part, we will use the Chebyshev bound to construct the Upper Confidence Bound. This will result in an algorithm that we will call Chebyshev-UCB. 

From the last discussion, we derived the following upper confidence bound for the mean:

$$ C_a(T_a(t),\delta)= \hat{\mu}_a(t)+\sqrt{\frac{\sigma^2}{T_a(t)\delta}},$$

where $\hat{\mu}_a$ is the current sample mean for arm $a$:

$$ \hat{\mu}_a =\frac{1}{T_a(t)} \sum_{i=1}^{T_a(t)} X_{a,i}$$

For this section, we will choose a time-varying $\delta$ to ensure that each arm will always be pulled:

$$ \delta(t)=\frac{1}{t^2},$$

resulting in the confidence bound:

$$C_a(T_a(t),\delta(t))= \begin{cases} \qquad \qquad \infty \ \qquad \qquad &: T_a(t)=0 \\ \hat{\mu}_a(t)+\sqrt{\frac{t^2\sigma^2}{T_a(t)}} \ \ \ \ \ \  &: T_a(t)>0\end{cases}$$

Using this form of the upper confidence bound, fill out the following function which returns the choice of arm as well as the upper confidence bounds of each arm.

In [ ]:
def chebyshev_pull_arm(t,variance,times_pulled,rewards):
    """ 
    Implement the choice of arm for the Chebyshev-UCB algorithm
    
    Inputs:
    t                  - iteration of the bandit algorithm
    variance           - variance of all the arms
    times_pulled       - a list of length K (where K is the number of arms) of the number of 
                         times each arm has been pulled
    rewards            - a list of K lists. Each of the K lists holds the samples received from pulling each arm up 
                         to iteration t. 
    
    Returns:
    arm                -  integer representing the arm that the chebyshev-UCB algorithm would choose.
    confidence bounds  -  a list of the chebyshev-dervied upper confidence bounds for each arm
    """
    K=len(times_pulled)
    delta=1.0/t**2
    
    confidence_bounds=[] #TODO
    
    arm=#TODO
    
    return arm , confidence_bounds

Given the function you have filled out, let us investigate the pseudo-regret of the chebyshev-UCB algorithm. Since the pseudo-regret is an expectation of the regret, the following cell runs the algorithm $20$ times anc computes the average pseudo-regret across all runs.

In [ ]:
# Define the figure and size of the figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

# Define the time horizon of each run, and the number of runs of each the algorithm.
T=1000
num_runs=20

# Initialize pseudo-regret
chebyshev_pseudo_regret=0
for runs in range(num_runs):
    # re-initialize bandit environment and counters
    bandit_env.initialize(make_plot=0)
    for t in range(1,T+1):
        # choose the arm using the chebyshev-UCB algorithm
        arm,confidence_bounds=chebyshev_pull_arm(t,variance,bandit_env.times_pulled,bandit_env.rewards)
        
        # pull the arm
        bandit_env.pull_arm(arm)
    
    #keep track of pseudo-regret
    chebyshev_pseudo_regret+=np.array(bandit_env.regret)

plt.plot(chebyshev_pseudo_regret/num_runs)
plt.xlabel('#TODO')
plt.ylabel('#TODO')
plt.show()


## B. UCB
We will now implement the classic version of the UCB algorithm which uses Chernoff/Hoeffding bounds. 

Recall that for the sample mean of $n$ Gaussian random variables with variance $\sigma^2$, the Chernoff bound is of the form:

$$ \mathbb{P}(\hat{\mu}_a-\mu_a > \epsilon) < e^{-\frac{n\epsilon^2}{2\sigma^2}}$$

This bound results in the upper confidence bound:

$$ C_a(T_a(t),\delta)= \hat{\mu}_a(t)+\sqrt{\frac{2\sigma^2}{T_a(t)}\log{\frac{1}{\delta}}},$$

where $\hat{\mu}_a$ is the current sample mean for arm $a$:

$$ \hat{\mu}_a =\frac{1}{T_a(t)} \sum_{i=1}^{T_a(t)} X_{a,i}$$

We will once again choose the same time-varying $\delta$ to ensure that we will always explore the arms.

$$ \delta(t)=\frac{1}{t^2},$$

resulting in the confidence bound:

$$C_a(T_a(t),\delta(t))= \begin{cases} \qquad \qquad \infty \ \qquad \qquad &: T_a(t)=0 \\ \hat{\mu}_a(t)+\sqrt{\frac{4\sigma^2}{T_a(t)}\log{t}} \ \ \ \ \ \  &: T_a(t)>0\end{cases}$$

Now, use this confidence bound to fill out the following function which returns the choice of arm as well as the upper confidence bounds of each arm.

In [ ]:
def UCB_pull_arm(t,variance,times_pulled,rewards):
    """ 
    Implement the choice of arm for the UCB algorithm
    
    Inputs:
    iteration          - iteration of the bandit algorithm
    times_pulled       - a list of length K (where K is the number of arms) of the number of 
                         times each arm has been pulled
    rewards            - a list of K lists. Each of the K lists holds the samples received from pulling each arm up 
                         to iteration t. 
    
    Returns:
    arm                -  integer representing the arm that the UCB algorithm would choose.
    confidence bounds  -  a list of the upper confidence bounds for each arm
    """

    K=len(times_pulled)
    delta=1.0/t**2
    
    confidence_bounds=[] #TODO
    
    arm=#TODO
    
    return arm , confidence_bounds

Given the function you have filled out, let us investigate the pseudo-regret of the UCB algorithm.

In [ ]:
#Initialize Figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

#Initialize pseudo-regret
UCB_pseudo_regret=0
for runs in range(num_runs):
    #Initialize Bandit_environment
    bandit_env.initialize(make_plot=0)
    for t in range(1,T+1):
        #Choose arm using UCB algorithm
        arm,confidence_bounds=UCB_pull_arm(t,variance,bandit_env.times_pulled,bandit_env.rewards)
        
        #Pull Arm
        bandit_env.pull_arm(arm)
        
    #Keep track of pseudo-regret  
    UCB_pseudo_regret+=np.array(bandit_env.regret)
    
#Make plot
plt.plot(UCB_pseudo_regret/num_runs)
plt.xlabel('#TODO')
plt.ylabel('#TODO')
plt.show()

## C. Compare the Algorithms
Let us now compare the regret of the UCB and Chebyshev-UCB algorithms

In [ ]:
#Make Plot
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

#Plot the pseudo-regrets of each algorithm
plt.plot(chebyshev_pseudo_regret/num_runs ,label='#TODO')
plt.plot(UCB_pseudo_regret/num_runs ,label='#TODO')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pseudo-Regret')
plt.show()

##  D. Dependence on the Sub-optimality Gap
Let us now compare the regret of the UCB algorithm on various instances with different minimum gaps between the optimal and sub-optimal arms. As we saw in Lecture, it should be much harder to find the best arm is the sub-optimal arms are very similar to it.

We define the gap $\Delta_a$ of a suboptimal arm $a=2,...,K-1$ as:

$$ \Delta_a=\mu_1-\mu_a$$

We denote the minimum gap as:

$$ \Delta=\min_{a>1} \Delta_a$$ 

The following cell runs the UCB algorithm you have implemented on 5 UCB instances with decreasing minimum gaps:

$$\Delta=[4,2,1,0.5,0.25]$$


In [ ]:
# Initialize Figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

# Defines the Gaps and means of each MAB instance we will be using
gaps=[4,2,1,0.5,0.25]
env_means=[[10,6,5,4,3,2],[10,8,7,6,5,4],[10,9,8,7,6,5],[10,9.5,8.5,7.5,6.5,5.5],[10,9.75,8.75,7.75,6.75,5.75]]


for Delta,means in zip(gaps,env_means):
    #Create bandit environment with the given means and standard deviations
    env=BanditEnv(means,standard_deviations)
    #Initialize Pseudo_regret
    pseudo_regret=0
    for run in range(num_runs):
        #Initialize Bandit environment between runs
        env.initialize(make_plot=0)
        for t in range(1,T+1):
            #Choose UCB arm
            arm,confidence_bounds=UCB_pull_arm(t,variance,env.times_pulled,env.rewards)
            #Aull arm
            env.pull_arm(arm)
        
        #Log pseudo-regret
        pseudo_regret+=np.array(env.regret)
    
    #Plot pseudo-regret for UCB on the given bandit instance
    plt.plot(pseudo_regret/num_runs,label=r'$\Delta$: {}'.format(Delta))

plt.legend()
plt.xlabel('Time')
plt.ylabel('Pseudo-Regret')
plt.show()

### Visualize Your Algorithm
If you want to visualize your algorithm, you can use the following interactive demo (If it is lagging, do not worry this part is not graded and is meant to build your intuition for the algorithm):

In [ ]:
plt.rcParams['figure.figsize']=[9,8]

# Creates an interactive bandit instance with an option to test your algorithm.
#     - Pull an arm by clickling on the colored button
#     - Allow your algorithm to choose the arm by clicking on the UCB button
#     - The true means of the distributions are shown with the dashed horizontal lines
#     - Large solid circle is the sample mean of the arm
#     - Solid vertical line is the upper confidence bound you have calculated
#     - The reward distribution of each arm is shown on the right and can be toggled on/off by checking the box
#     - Running Pseudo-regret is shown on the bottom and can be toggled on/off by checking the box

# You may need to rerun this cell to restart the gui
alg=Interactive_UCB_Algorithm(bandit_env,UCB_pull_arm,'UCB')
alg.run_Interactive_Alg()

# 2. The Bayesian Approach: Thompson Sampling
The third algorithm we will analyze is the Bayesian take on multi-armed bandits, known as Thompson Sampling. In this setting, you begin with a prior over the mean of each arm $\pi_a(\mu_a)$.

At each round $t=1,2...$, the algorithm computes the posterior probability $p_{a,t}$ that arm $a\in \mathcal{A}$ has the highest mean reward:

$$ p_{a,t}=\mathbb{P}\bigg(\mu_a=\max_{a'} \mu_{a'} \ \bigg| \ X_{1,A_1},...,X_{t-1,A_{t-1}}\bigg).$$

The choice of arm is then randomly sampled from the distribution $p_t$ over $\mathcal{A}$, where each arm $a\in \mathcal{A}$ has probsbility $p_{a,t}$:

$$ A_t \sim p_t $$ 

## Implementing Thompson Sampling

Since the posterior distribution over each arm having the maximum mean is often intractable to compute, in practice we often implement a simpler algorithm that nevertheless accomplishes the same task.

At each rount $t=1,2....$, you keep track of the posterior distribution over $\mu_a$, for each arm $a \in \{ 0,1,...,K-1\}$,  given all the samples you have observed from that arm $X_{a,1},...,X_{a,T_a(t-1)}$:

$$P_{a,t}=\mathbb{P}(\mu_a | X_{a,1},...,X_{a,T_a(t-1)}).$$

You then take one sample from $P_{a,t}$ and choose the arm with the highest sample:

1. $\mu_{a,t} \sim  P_{a,t}$ for $a \in \{0,1,...,K-1\}$.
2. Choose arm:
$$ A_t=\underset{a \in \{ 0,1,...,K-1\}}{\operatorname{argmax}} \mu_{a,t}$$

Since the reward distributions in this lab are Gaussians with known variance $\sigma_a^2$, we know from our investigation of conjugate priors that if we have Gaussian priors: $\pi_a(\mu_a)=\mathcal{N}(\mu_{a,0},\sigma_{a,0}^2)$, the posterior distribution for each arm will also be a Gaussian. 

Therefore, to implement Thompson Sampling in this lab, the posterior distributions for each arm in this lab at each time $t=1,2,...$ are given by:

$$ P_{a,t}=\mathcal{N}(\hat\mu_{a,t},\hat{\sigma}_{a,t}^2)$$

where,
 $$\hat{\sigma}_{a,t}^2 =\bigg(\frac{1}{\sigma_{a,0}^2}+\frac{T_a(t-1)}{\sigma_a^2}\bigg)^{-1} $$
$$ \hat\mu_{a,t}=\hat{\sigma}_{a,t}^2 \bigg( \frac{\mu_{a,0}}{\sigma_{a,0}^2}+\frac{\sum_{i=1}^{T_a(t-1)} X_{a,i}}{\sigma_a^2} \bigg)$$

Fill out the following function that implements the choice of arm for the Thompson Sampling algorithm with Gaussian arms and prior. 

In [ ]:
def TS_pull_arm(t,variance,times_pulled,rewards,prior_means,prior_variances):
    """ 
    Implement the choice of arm for the Thompson Sampling Algorithm when the arms and priors are Gaussians.
    
    Inputs:
    iteration          - iteration of the bandit algorithm.
    times_pulled       - a list of length K (where K is the number of arms) of the number of 
                         times each arm has been pulled.
    rewards            - a list of K lists. Each of the K lists holds the samples received from pulling each arm up 
                         to iteration t.
    prior_means        - a list of length K with the mean of the priorsfor each arm.
    prior_variances    - a list of length K with the variance of the prior for each arm.
    
    Returns:
    arm                - integer representing the arm that the UCB algorithm would choose.
    posterior_samples  - list of samples from the posterior used to choose the arm. 
    posterior_means    - list of means of the posterior for each arm
    posterior_vars     - list of variances of the posteriors of each arm
    """

    K=len(times_pulled)
    
    posterior_samples=[] #TODO
    posterior_means=[]   #TODO
    posterior_vars=[]    #TODO       
    arm=                 #TODO
    
    return arm , posterior_samples , posterior_means,posterior_vars

## A. Thompson Sampling with Good Priors
As we saw in class, the performance of Thompson Sampling can vary drastically with the quality of the prior. 

First, let us analyze the performance of Thompson Sampling when the priors reflect the correct rankings of the arms (meaning that the prior mean for arm $0$ is the highest). We will compare it to the performance of the UCB algorithm.

In [ ]:
#Initialize Figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

#Define Prior Means and Variances
prior_means=[12,9,8,7,4,3,2]
prior_vars=[2.2,2.2,2.2,2.2,2.2,2.2]

#Initialize pseudo-regret
TS_pseudo_regret=0
for runs in range(num_runs):
    #Initialize bandit environment
    bandit_env.initialize(make_plot=0)
    for t in range(1,T+1):
        #Choose arm with Thompson Sampling
        arm,samples,means,variances=TS_pull_arm(t,variance,bandit_env.times_pulled,bandit_env.rewards,prior_means,prior_vars)
        
        #Pull Arm
        bandit_env.pull_arm(arm)
    
    #Keep track of regret Regret
    TS_pseudo_regret+=np.array(bandit_env.regret)
    
#Plot Thompson Sampling vs. UCB regret
plt.plot(TS_pseudo_regret/num_runs ,label='TS Regret')
plt.plot(UCB_pseudo_regret/num_runs ,label='UCB Regret')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pseudo-Regret')
plt.show()

## B. Thompson Sampling with Bad Priors
Now let us analyze the performance of Thompson Sampling when the priors have completely incorrect correct rankings of the arms, meaning that the prior mean for arm $0$ is the lowest.

In [ ]:
#Initialize Figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

#Define prior means and standard deviations
prior_means=[2,3,4,5,6,7]
prior_vars=[2.2,2.2,2.2,2.2,2.2,2.2]

#Initialize pseudo-regret
TS_pseudo_regret=0
for runs in range(num_runs):
    #Initialize bandit environment
    bandit_env.initialize(make_plot=0)
    for t in range(1,T+1):
        #Chosoe arm with Thompson Sampling
        arm,samples,means,variances=TS_pull_arm(t,variance,bandit_env.times_pulled,bandit_env.rewards,prior_means,prior_vars)
        
        #Pull Arm
        bandit_env.pull_arm(arm)
    
    #Keep track of regret Regret
    TS_pseudo_regret+=np.array(bandit_env.regret)
    
#Plot Thompson Sampling vs. UCB regret
plt.plot(TS_pseudo_regret/num_runs ,label='TS Regret')
plt.plot(UCB_pseudo_regret/num_runs ,label='UCB Regret')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pseudo-Regret')
plt.show()

## B. Thompson Sampling with the same prior for each arm
Now let us analyze the performance of Thompson Sampling when the priors are the same for all arms.

In [ ]:
#Make Figure
plt.rcParams['figure.figsize']=[9,4]
plt.figure()

#Define prior means and variances
prior_means=[8,8,8,8,8,8]
prior_vars=[2.5,2.5,2.5,2.5,2.5,2.5]

#Initialize pseudo-regret
TS_pseudo_regret=0
for runs in range(num_runs):
    #Initialize bandit environment
    bandit_env.initialize(make_plot=0)
    for t in range(1,T+1):
        #Chosoe arm with Thompson Sampling
        arm,samples,means,variances=TS_pull_arm(t,variance,bandit_env.times_pulled,bandit_env.rewards,prior_means,prior_vars)
        
        #Pull Arm
        bandit_env.pull_arm(arm)
    
    #Keep track of regret Regret
    TS_pseudo_regret+=np.array(bandit_env.regret)
    
#Plot Thompson Sampling vs. UCB regret
plt.plot(TS_pseudo_regret/num_runs ,label='TS Regret')
plt.plot(UCB_pseudo_regret/num_runs ,label='UCB Regret')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pseudo-Regret')
plt.show()

### Visualize Your Algorithm
If you want to visualize your algorithm, you can use the following interactive demo (If it is lagging, do not worry this part is not graded and is meant to build your intuition for the algorithm):

In [ ]:
plt.rcParams['figure.figsize']=[9,8]

# Creates an interactive bandit instance with an option to test your algorithm.
#     - Pull an arm by clickling on the colored button.
#     - Allow your algorithm to choose the arm by clicking on the UCB button.
#     - The true means of the distributions are shown with the dashed horizontal lines.
#     - Large solid circle is the sample mean of the rewards for the arm.
#     - Solid vertical line shows the 95% credible interval for the arm.
#     - The reward distribution of each arm is shown on the right and can be toggled on/off by checking the box.
#     - Running Pseudo-regret is shown on the bottom and can be toggled on/off by checking the box.

# You may need to rerun this cell to restart the gui
alg=Interactive_TS_Algorithm(bandit_env,TS_pull_arm,'TS',prior_means,prior_vars)
alg.run_Interactive_Alg()

## 3. Pros and Cons of UCB and Thompson Sampling

In the following cell, write a few sentences comparing and contrasting Chebyshev-UCB, UCB, and Thompson Sampling. What are some pros and cons of UCB and of Thompson Sampling?

#TODO